# Sparse Non-Negative PARAFAC

This notebook is based on [sparse_demo.ipynb](sparse_demo.ipynb#parafac). 

As before, we start with a random sparse tensor, constructed so that it has a tensor factorization of rank 5.

Because non-negative PARAFAC can take longer to converge than non-masked PARAFAC and also produce dense factors, we will use a smaller tensor than in the other notebook.

In [1]:
shape = (1000, 1001, 1002)
rank = 5

import sparse
starting_weights = sparse.ones(rank)
starting_factors = [sparse.random((i, rank)) for i in shape]
starting_factors

[<COO: shape=(1000, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1001, 5), dtype=float64, nnz=50, fill_value=0.0>,
 <COO: shape=(1002, 5), dtype=float64, nnz=50, fill_value=0.0>]

In [2]:
from tensorly.contrib.sparse.cp_tensor import cp_to_tensor
tensor = cp_to_tensor((starting_weights, starting_factors))
tensor

Format,coo
Data Type,float64
Shape,"(1000, 1001, 1002)"
nnz,4884
Density,4.869382114891097e-06
Read-only,True
Size,152.6K
Storage ratio,0.0


In [3]:
def format_size(size_bytes):
    size = size_bytes
    for unit in ['B', 'KiB', 'MiB', 'GiB', 'TiB']:
        if not int(size/1024):
            return f'{round(size)}.{unit}'
        else:
            size /= 1024

In [4]:
format_size(tensor.nbytes)             # Actual memory usage in GB

'153.KiB'

In [5]:
import numpy as np
format_size(np.prod(tensor.shape) * 8)  # Memory usage if array was dense, in GB

'7.GiB'

Now we factor the tensor. Note that at this time, you have to use the `non_negative_parafac` function from the sparse backend when using a sparse mask to avoid memory blowups.

In [6]:
import time
%load_ext memory_profiler
from tensorly.contrib.sparse.decomposition import non_negative_parafac

In [7]:
%%memit
start_time = time.time()
sparse_kruskal = non_negative_parafac(tensor, rank=rank, init='random', verbose=True)
end_time = time.time()
total_time = end_time - start_time
print('Took %d mins %d secs' % (divmod(total_time, 60)))

reconstruction error=0.943207231771627
iteration 1, reconstraction error: 0.758341832872278, decrease = 0.18486539889934905
iteration 2, reconstraction error: 0.6880274843697167, decrease = 0.07031434850256124
iteration 3, reconstraction error: 0.5399027985471302, decrease = 0.14812468582258653
iteration 4, reconstraction error: 0.48988338684888716, decrease = 0.05001941169824303
iteration 5, reconstraction error: 0.48772494556981577, decrease = 0.002158441279071388
iteration 6, reconstraction error: 0.4864308244191075, decrease = 0.0012941211507082606
iteration 7, reconstraction error: 0.48555333649559845, decrease = 0.0008774879235090571
iteration 8, reconstraction error: 0.48491546799409857, decrease = 0.0006378685014998831
iteration 9, reconstraction error: 0.4844427370307753, decrease = 0.0004727309633232868
iteration 10, reconstraction error: 0.484098425435411, decrease = 0.00034431159536429945
iteration 11, reconstraction error: 0.4838552615479818, decrease = 0.00024316388742917

Let's look at one of the values that was masked out.

In [8]:
tensor.coords.T[0]

array([  8, 117, 198])

In [9]:
orig_val = tensor[tuple(tensor.coords.T[0])]
orig_val

0.2240418375433051

See the [sparse_demo.ipynb](sparse_demo.ipynb) for how to calculate individual values from the factors. Note that we do not compare the entire tensor because it would be dense, and memory usage would be prohibitive.

In [10]:
weights, factors = sparse_kruskal
computed_val = np.sum(np.prod(sparse.stack([factors[i][idx] for i, idx in enumerate(tuple(tensor.coords.T[0]))], 0), 0))
computed_val

6.097320102631128e-49

In [11]:
np.abs(orig_val - computed_val)

0.2240418375433051

In [12]:
[f.density for f in factors]

[1.0, 1.0, 1.0]